In [ ]:
pip install sendgrid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 3.6 MB/s 
  Created wheel for starkbank-ecdsa: filename=starkbank_ecdsa-2.2.0-py3-none-any.whl size=15987 sha256=c39048f9dd522417422347ec9cc6fc6bef66173cc95158a472624dd12f3b9130
  Stored in directory: /root/.cache/pip/wheels/ae/84/09/5c4abe28789e5a3e7d908ccfdbc681c2e0c2eb41b82f4a8a67
Successfully built starkbank-ecdsa


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Crawler to Extract Email Address from Specific Website

In [ ]:
def is_valid_url(url):
    import re
    regex = re.compile(
        r'^https?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'  # domain...
        r'localhost|'  # localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return url is not None and regex.search(url)

In [ ]:
import re
import requests
import requests.exceptions
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup

# starting url
starting_url = 'https://skoltech.ru'
based_url = "https://faculty.skoltech.ru"
# a queue of urls to be crawled
unprocessed_urls = deque([starting_url])

# set of already crawled urls for email
processed_urls = set()

# a set of fetched emails
emails = set()
phones = set()

# process urls one by one from unprocessed_url queue until queue is empty
while len(unprocessed_urls):
    print(unprocessed_urls)
    # move next url from the queue to the set of processed urls
    url = unprocessed_urls.popleft()
    if not is_valid_url(url): continue
    # if not (url.startswith("https://rdkb.ru/") or url.startswith("http://rdkb.ru/")): continue
    # if not (url.startswith(based_url)): continue
    if "javascript" in url or "mailto" in url or "tel:" in url or ".jpg" in url: continue
    processed_urls.add(url)

    # extract base url to resolve relative links
    parts = urlsplit(url)
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    # path = url[:url.rfind('/')+1] if '/' in parts.path else url
    path = based_url
    # get url's content
    print("Crawling URL %s" % url)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
        # ignore pages with errors and continue with next url
        continue

    # extract all email addresses and add them into the resulting set
    # You may edit the regular expression as per your requirement
    new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
    new_phones = set(re.findall(r"^\s*(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})(?: *x(\d+))?\s*$", response.text, re.I))
    emails.update(new_emails)
    phones.update(new_phones)
    print(new_emails)
    print(new_phones)
    # create a beutiful soup for the html document
    soup = BeautifulSoup(response.text, 'lxml')

    # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
    for anchor in soup.find_all("a"):
        # extract link url from the anchor
        link = anchor.attrs["href"] if "href" in anchor.attrs else ''
        # resolve relative links (starting with /)
        if link.startswith('/'):
            link = base_url + link
        elif not link.startswith('http'):
            link = path + link
        # add the new url to the queue if it was not in unprocessed list nor in processed list yet
        if not link in unprocessed_urls and not link in processed_urls:
            unprocessed_urls.append(link)

In [ ]:
 print(emails)

# Bulk mailing using Sendgrid API (retired in 2023)


In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
import csv

In [ ]:
SENDGRID_API_KEY="insert or import your API key here

TEMPLATE_PATH = "/campaigns/sendgrid_test/template1.txt"
CONTACTS_PATH = "/campaigns/sendgrid_test/verified_emails_1412_60k-100k.csv"
CONTACT_START = 0
SEND_NUMBER = 1 # number of contacts to send to

In [ ]:
with open(TEMPLATE_PATH, 'r') as file:
    template = str(file.read().rstrip())

In [ ]:
sg = sendgrid.SendGridAPIClient(SENDGRID_API_KEY)

In [ ]:
# reading through each row of csv file and sending mail separatedly
from_email = ("opros@vktsoom.ru","Фонд Общественное Мнение ВКЦООМ")
subject = "Участвуйте во Всероссийском Социологическом Опросе ВКЦООМ-Циф 2022"

start = CONTACT_START
number = SEND_NUMBER

with open(CONTACTS_PATH) as file_obj:
    reader_obj = csv.reader(file_obj)
    for row in reader_obj:
      # skip first number of contacts
      if start > 0:
        start = start - 1
      else:
        if number == 0: break
        # keep track of how many contacts have sent to
        number = number - 1;
        # customizing email
        recipient = "tuftsforukraine@gmail.com"
        url = row[1]
        print(number,"SENDING TO: ",recipient)
        content = template.replace("{{{recipient}}}",recipient)
        content = content.replace("{{{button_url}}}",url)
        to_email = To(recipient) # change here
        content = Content("text/html", content) #change here
        mail = Mail(from_email, to_email, subject, content)
        mail.add_header(Header("Priority", "Urgent"));
        mail.add_header(Header("Importance", "high"));
        response = sg.client.mail.send.post(request_body=mail.get())
        print(response.status_code)
        print(response.body)
        print(response.headers)
        print(number,"=====================COMPLETED")

0 SENDING TO:  test@gmail.com
202
b''
Server: nginx
Date: Tue, 20 Dec 2022 04:26:06 GMT
Content-Length: 0
Connection: close
X-Message-Id: 3NFN2z_qTgybBybZebhL9Q
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=600; includeSubDomains


0 =====================COMPLETED


In [ ]:
from ast import literal_eval
params = {'start_date': '2022-12-15'}

# retrieve stats by mailbox provider
# response = sg.client.mailbox_providers.stats.get(
#     query_params=params
# )

# by device type (not useful)
# response = sg.client.devices.stats.get(
#     query_params=params
# )

# global stats
response = sg.client.stats.get(
    query_params=params
)

res = literal_eval(response.body.decode('ascii'))
for i in res:
  print("date", i['date'])
  for j in i['stats']:
      print(j)

# 9000 left
# do not sent too much immediately to avoid deferral

date 2022-12-15
{'metrics': {'blocks': 0, 'bounce_drops': 0, 'bounces': 0, 'clicks': 0, 'deferred': 0, 'delivered': 0, 'invalid_emails': 0, 'opens': 0, 'processed': 0, 'requests': 0, 'spam_report_drops': 0, 'spam_reports': 0, 'unique_clicks': 0, 'unique_opens': 0, 'unsubscribe_drops': 0, 'unsubscribes': 0}}
date 2022-12-16
{'metrics': {'blocks': 1, 'bounce_drops': 0, 'bounces': 2, 'clicks': 23, 'deferred': 0, 'delivered': 123, 'invalid_emails': 0, 'opens': 19, 'processed': 126, 'requests': 126, 'spam_report_drops': 0, 'spam_reports': 0, 'unique_clicks': 8, 'unique_opens': 9, 'unsubscribe_drops': 0, 'unsubscribes': 0}}
date 2022-12-17
{'metrics': {'blocks': 915, 'bounce_drops': 0, 'bounces': 930, 'clicks': 133, 'deferred': 65412, 'delivered': 77011, 'invalid_emails': 2, 'opens': 938, 'processed': 78843, 'requests': 78845, 'spam_report_drops': 0, 'spam_reports': 1, 'unique_clicks': 66, 'unique_opens': 826, 'unsubscribe_drops': 0, 'unsubscribes': 0}}
date 2022-12-18
{'metrics': {'blocks':

In [ ]:
response = sg.client.user.credits.get()
print(response.status_code)
print(response.body)
print(response.headers)

# bounces

# response = sg.client.suppression.bounces.get()
# print(len(literal_eval(response.body.decode('ascii'))))
# # blocks
# response = sg.client.suppression.blocks.get()
# print(len(literal_eval(response.body.decode('ascii'))))
# spam
# response = sg.client.suppression.spam_reports.get()
# print(len(literal_eval(response.body.decode('ascii'))))

# res = literal_eval(response.body.decode('ascii'))
# for i in res:
#   print(i)


200
b'{"is_hard_limit":false,"last_reset":"2022-12-01","next_reset":"2023-01-01","overage":0,"remain":93,"reset_frequency":"monthly","total":100000,"used":99907}'
Server: nginx
Date: Tue, 20 Dec 2022 04:13:31 GMT
Content-Type: application/json
Content-Length: 155
Connection: close
access-control-allow-methods: HEAD, OPTIONS, GET
access-control-max-age: 21600
access-control-expose-headers: Link, Location
access-control-allow-origin: *
access-control-allow-headers: AUTHORIZATION, Content-Type, On-behalf-of, x-sg-elas-acl, X-Recaptcha, X-Request-Source, Browser-Fingerprint
content-security-policy: default-src https://api.sendgrid.com; frame-src 'none'; object-src 'none'
x-content-type-options: nosniff
strict-transport-security: max-age=31536000
x-client-ff: 1000
x-ratelimit-remaining: 598
x-ratelimit-limit: 600
x-ratelimit-reset: 1671509640
twilio-request-id: RQ2JA8dDQSHdcQkQDs8lyuiMY5kHt
x-envoy-upstream-service-time: 333
Powered-By: Mako
Strict-Transport-Security: max-age=600; includeSu